In [3]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [4]:
#and yaml file
with open('data.yaml','r') as f:
    data_yaml = yaml.load(f,Loader=SafeLoader)
labels = data_yaml['names']
print(labels)

['person', 'car', 'chair', 'plant', 'bottle']


In [5]:
#load yolo model 
yolo = cv2.dnn.readNetFromONNX('./Model6/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [6]:
#load image and get the yolo predictions from the image
img = cv2.imread('./streetTraffic.jpg')
image = img.copy()
row,col,d = image.shape
#get the yolo predection from image
#convert our image to a square matrix
max_rc = max(row,col)
input_image = np.zeros((max_rc,max_rc,d), dtype=np.uint8)
input_image[0:row,0:col] = image
#step2 get predection from square array
INPUT_WIDTH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WIDTH_YOLO,INPUT_WIDTH_YOLO),swapRB=True,crop=False)
yolo.setInput(blob)
preds = yolo.forward() #detection or predection from yolo
print(preds.shape)

(1, 25200, 10)


In [22]:
#non Maximum Suppression
#step1 : filter detection based on confiide,ce (0.4) and probability 0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []
# width and height of the image input_image
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WIDTH_YOLO
y_factor = image_h/INPUT_WIDTH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] #confidence of detection an object 
    if confidence > 0.4:
        class_score = row[5:].max() #maximum probability from 5 object
        class_id = row[5:].argmax() # get index position at which maximum probability
        if class_score > 0.25:
            cx,cy,w,h = row[0:4]
            #construct bounding from four values
            #left, top,width and heigt 
            left = int((cx-0.5*w)*x_factor)
            top = int((cy-0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)
            box = np.array([left,top,width,height])
            #append values into the lists 
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

#clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

#NMS
index = cv2.dnn.NMSBoxes(boxes_np,confidences_np,0.25,0.45).flatten()


In [23]:
index

array([3, 2], dtype=int32)

In [26]:
#Draw the BoundingBox
for ind in index:
    x,y,w,h = boxes_np[ind]
    bb_conf = int(confidences_np[ind]*100)
    classes_id = classes[ind]
    class_name = labels[classes_id]
    text = f'{class_name}: {bb_conf}%'
    print(text)
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.rectangle(image,(x,y-30),(x+w,y),(255,255,255),-1)
    cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)

car: 65%
person: 63%


In [27]:
cv2.imshow('original',img)
cv2.imshow('yolo_predection',image)
cv2.waitKey(0)
cv2.destroyAllWindows()